In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_mal
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/transliterated/'

    train_data = read_task(dataset_location , split = 'mal_train_trans')
    dev_data = read_task(dataset_location , split = 'mal_dev_trans')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_mal
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['xlm-roberta-base']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Malayalam' + 'transliterated' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 79 of 497
FINSIHED BATCH: 99 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 0.7248393297195435
DEV ACC: 0.5602678571428571
DEV Precision: 0.423917595208918
DEV Recall: 0.42837569962348804
DEV F1Score: 0.3921767143227927
BEST ACCURACY -->  DEV: 0.56027
BEST PRECISION -->  DEV: 0.42392
BEST RECALL -->  DEV: 0.42838
BEST F1SCORE -->  DEV: 0.39218
TIME PER EPOCH: 12.30593375

FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 0.7215909957885742
DEV ACC: 0.7180059523809524
DEV Precision: 0.6458050223908111
DEV Recall: 0.6408296759846129
DEV F1Score: 0.6225165615346369
BEST ACCURACY -->  DEV: 0.71801
BEST PRECISION -->  DEV: 0.65412
BEST RECALL -->  DEV: 0.64083
BEST F1SCORE -->  DEV: 0.62252
TIME PER EPOCH: 12.463839948177338

Training epoch: 10
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 79 of 497
FINSIHED BATCH: 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 1.0144116878509521
DEV ACC: 0.5892857142857143
DEV Precision: 0.37170599409252114
DEV Recall: 0.4516824787898842
DEV F1Score: 0.3942743039412897
BEST ACCURACY -->  DEV: 0.58929
BEST PRECISION -->  DEV: 0.37171
BEST RECALL -->  DEV: 0.45168
BEST F1SCORE -->  DEV: 0.39427
TIME PER EPOCH: 12.454740965366364

Training epoch: 2
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 49

...

Training epoch: 1
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 79 of 497
FINSIHED BATCH: 99 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 0.9460113644599915
DEV ACC: 0.5755208333333333
DEV Precision: 0.42914920931445033
DEV Recall: 0.44686326987548786
DEV F1Score: 0.4077427037414921
BEST ACCURACY -->  DEV: 0.57552
BEST PRECISION -->  DEV: 0.42915
BEST RECALL -->  DEV: 0.44686
BEST F1SCORE -->  DEV: 0.40774
TIME PER EPOCH: 12.444334

FINSIHED BATCH: 99 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 1.0408838987350464
DEV ACC: 0.716703869047619
DEV Precision: 0.6518366366643884
DEV Recall: 0.6261670050335189
DEV F1Score: 0.6160462989933626
BEST ACCURACY -->  DEV: 0.72173
BEST PRECISION -->  DEV: 0.65545
BEST RECALL -->  DEV: 0.62617
BEST F1SCORE -->  DEV: 0.61605
TIME PER EPOCH: 12.437533684571584

Training epoch: 10
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 7

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 79 of 497
FINSIHED BATCH: 99 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159 of 497
FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 1.058260202407837
DEV ACC: 0.5671502976190477
DEV Precision: 0.38409360153811184
DEV Recall: 0.4138545312808292
DEV F1Score: 0.3772121514803522
BEST ACCURACY -->  DEV: 0.56715
BEST PRECISION -->  DEV: 0.38409
BEST RECALL -->  DEV: 0.41385
BEST F1SCORE -->  DEV: 0.37721
TIME PER EPOCH: 12.46391600

FINSIHED BATCH: 179 of 497
FINSIHED BATCH: 199 of 497
FINSIHED BATCH: 219 of 497
FINSIHED BATCH: 239 of 497
FINSIHED BATCH: 259 of 497
FINSIHED BATCH: 279 of 497
FINSIHED BATCH: 299 of 497
FINSIHED BATCH: 319 of 497
FINSIHED BATCH: 339 of 497
FINSIHED BATCH: 359 of 497
FINSIHED BATCH: 379 of 497
FINSIHED BATCH: 399 of 497
FINSIHED BATCH: 419 of 497
FINSIHED BATCH: 439 of 497
FINSIHED BATCH: 459 of 497
FINSIHED BATCH: 479 of 497
Validation loss per 100 evaluation steps: 0.939992368221283
DEV ACC: 0.7191220238095238
DEV Precision: 0.6467673645300039
DEV Recall: 0.6462141738711494
DEV F1Score: 0.6271855516365036
BEST ACCURACY -->  DEV: 0.72712
BEST PRECISION -->  DEV: 0.66146
BEST RECALL -->  DEV: 0.6498
BEST F1SCORE -->  DEV: 0.62759
TIME PER EPOCH: 12.431549521287282

Training epoch: 10
FINSIHED BATCH: 19 of 497
FINSIHED BATCH: 39 of 497
FINSIHED BATCH: 59 of 497
FINSIHED BATCH: 79 of 497
FINSIHED BATCH: 99 of 497
FINSIHED BATCH: 119 of 497
FINSIHED BATCH: 139 of 497
FINSIHED BATCH: 159

In [6]:
print(best_dev_acc)

0.7291666666666667


In [7]:
print(best_epoch)

9


In [8]:
print(best_dev_precision)

0.6626615154557715


In [9]:
print(best_dev_recall)

0.658351291977468


In [10]:
print(best_dev_f1score)

0.638579905944784


In [11]:
print(overall_list_dev_acc)

[[0.5602678571428571, 0.6711309523809524, 0.6787574404761905, 0.7003348214285714, 0.7036830357142857, 0.7075892857142857, 0.712797619047619, 0.7122395833333333, 0.7180059523809524, 0.6975446428571429], [0.5892857142857143, 0.6276041666666667, 0.6850818452380952, 0.6943824404761905, 0.7157738095238095, 0.7146577380952381, 0.7202380952380952, 0.7239583333333333, 0.7222842261904762, 0.7241443452380952], [0.5872395833333333, 0.6568080357142857, 0.6923363095238095, 0.6899181547619048, 0.6867559523809524, 0.7235863095238095, 0.6994047619047619, 0.7248883928571429, 0.71875, 0.7131696428571429], [0.5755208333333333, 0.6709449404761905, 0.6873139880952381, 0.7085193452380952, 0.6999627976190476, 0.7057291666666667, 0.7062872023809524, 0.7217261904761905, 0.716703869047619, 0.7183779761904762], [0.5671502976190477, 0.6558779761904762, 0.6614583333333333, 0.7023809523809524, 0.7126116071428571, 0.7178199404761905, 0.7271205357142857, 0.7113095238095238, 0.7191220238095238, 0.7291666666666667]]


In [12]:
print(overall_list_dev_precision)

[[0.423917595208918, 0.537318718720926, 0.5566165362139212, 0.6109689178701784, 0.6033791895983336, 0.6541192973631608, 0.6473338724943725, 0.6510084269913644, 0.6458050223908111, 0.6324273489990403], [0.37170599409252114, 0.4991877773836807, 0.5972349879687197, 0.6108218750019944, 0.6508529634141332, 0.6396597712626642, 0.6685742718906972, 0.6729838671525271, 0.6722086649051002, 0.6545781272882113], [0.38842541751549114, 0.5284843203092178, 0.591749637903407, 0.5937699088959594, 0.6140315266709598, 0.641104758290876, 0.6418931659202143, 0.6384328496458087, 0.6442762969613989, 0.6509904666995003], [0.42914920931445033, 0.547692036543485, 0.5717289579692139, 0.6216465782117042, 0.6000499624383554, 0.6200094044986723, 0.6198126477848528, 0.6554513610744263, 0.6518366366643884, 0.6517326462697703], [0.38409360153811184, 0.5158571650082249, 0.5653165048377342, 0.6342171102584083, 0.6400730108476955, 0.6455184461873633, 0.6614597360306531, 0.6484505624073639, 0.6467673645300039, 0.662661515

In [13]:
print(overall_list_dev_recall)

[[0.42837569962348804, 0.5497722758438426, 0.5985684236544266, 0.6264520374729144, 0.6173566542001415, 0.6325938324047568, 0.5962074819039381, 0.6343243369608915, 0.6408296759846129, 0.6560435226683914], [0.4516824787898842, 0.5746324335616959, 0.5572140595441957, 0.5927275079765683, 0.6267614482418843, 0.6208196314656399, 0.6387134167478179, 0.6383652448642013, 0.6571338998886267, 0.655148716246405], [0.4525252502086592, 0.5379968647071146, 0.5924198293099293, 0.6018509744024589, 0.6325617028571334, 0.6040460326260303, 0.636613650045345, 0.6266704645183127, 0.6318611286297735, 0.6377285313892456], [0.44686326987548786, 0.5522234294051936, 0.6038713583366112, 0.5945400540905847, 0.6056717162542847, 0.6140540650864487, 0.6246523540214455, 0.6213558266055775, 0.6261670050335189, 0.6362136236827677], [0.4138545312808292, 0.5608404885655343, 0.6033587970444587, 0.6084535817532142, 0.6087278556067256, 0.6274962057229649, 0.6288909058670088, 0.649802178462478, 0.6462141738711494, 0.658351291

In [14]:
print(overall_list_dev_f1score)

[[0.3921767143227927, 0.5183037645379232, 0.5611584083149738, 0.603868970060242, 0.5908997621961518, 0.6150544441225979, 0.592575380097978, 0.6196197512531738, 0.6225165615346369, 0.617849059712519], [0.3942743039412897, 0.5094086441532287, 0.5488014466920653, 0.579845857535442, 0.6128457720685444, 0.6091357676817312, 0.6303482252717558, 0.6302327549245658, 0.6443531147748602, 0.6327513175464891], [0.4023896624540371, 0.5138364711306104, 0.5654180624733283, 0.5792416936438621, 0.5972089383165257, 0.6014151397896893, 0.6172924239350396, 0.6147462683916957, 0.6173743845874636, 0.6275725460616127], [0.4077427037414921, 0.5268415680480298, 0.5693617472042609, 0.5815503173317352, 0.5829702746581507, 0.5957475658247197, 0.6016262672539948, 0.6121055678559936, 0.6160462989933626, 0.6217562330162966], [0.3772121514803522, 0.5232294489905637, 0.5608367656763502, 0.5986393253492046, 0.6014190081939994, 0.6111668637654075, 0.6203434801348423, 0.6275939589095446, 0.6271855516365036, 0.638579905944

In [15]:
#The best model is 0